<a href="https://colab.research.google.com/github/Toon-nooT/notebooks/blob/main/Donut_vs_pix2struct_2_Ghega_donut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we finetune the Donut model on the dataset prepared in notebook *'Donut vs pix2struct: 1 Ghega data prep.ipynb'*

See my [article](https://towardsdatascience.com/ocr-free-document-data-extraction-with-transformers-1-2-b5a826bc2ac3) for details

based on excellent [tutorial](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Donut/CORD/Fine_tune_Donut_on_a_custom_dataset_(CORD)_with_PyTorch_Lightning.ipynb) of Niels Rogge

## Download and extract dataset

In [ ]:
!wget 'https://huggingface.co/datasets/to-be/ghega_dataset_preprocessed/resolve/main/Ghega_dataset_preprocessed.zip'

--2023-06-08 11:10:06--  https://huggingface.co/datasets/to-be/ghega_dataset_preprocessed/resolve/main/Ghega_dataset_preprocessed.zip
Resolving huggingface.co (huggingface.co)... 18.155.68.44, 18.155.68.38, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/35/59/3559008cd39cee966e2682c53eec8e913f4e99ada42dba526b57607c7c03e478/c81de0c2ba98a3e8dde13841107a0b34ea77eccd9096e097f29e2db9c1305d62?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Ghega_dataset_preprocessed.zip%3B+filename%3D%22Ghega_dataset_preprocessed.zip%22%3B&response-content-type=application%2Fzip&Expires=1686481806&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzM1LzU5LzM1NTkwMDhjZDM5Y2VlOTY2ZTI2ODJjNTNlZWM4ZTkxM2Y0ZTk5YWRhNDJkYmE1MjZiNTc2MDdjN2MwM2U0NzgvYzgxZGUwYzJiYTk4YTNlOGRkZTEzODQxMTA3YTBiMzRlYTc3ZWNjZDkwOTZlMDk3Zj

In [ ]:
!unzip -q /content/Ghega_dataset_preprocessed.zip # -d /content/

## Set-up environment


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git datasets sentencepiece
!pip install -q wandb
!pip install  pytorch-lightning==1.9.5    #not using latest 2 version, otherwise: NotImplementedError: Support for validation_epoch_end has been removed in v2.0.0. DonutModelPLModule implements this method. You can use the on_validation_epoch_end hook instead.

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2

## Load model and processor


In [ ]:
from transformers import VisionEncoderDecoderConfig

max_length = 768
image_size = [1920, 1280]

config = VisionEncoderDecoderConfig.from_pretrained("naver-clova-ix/donut-base")
config.encoder.image_size = image_size # (height, width)
# update max_length of the decoder (for generation)
config.decoder.max_length = max_length

Next, we instantiate the model with our custom config, ass well as the processor.

In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel, BartConfig

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base", config=config)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


## Create PyTorch dataset and loader



In [ ]:
import json
import random
from typing import Any, List, Tuple

import torch
from torch.utils.data import Dataset

added_tokens = []

class DonutDataset(Dataset):
    """
    DonutDataset which is saved in huggingface datasets format. (see details in https://huggingface.co/docs/datasets)
    Each row, consists of image path(png/jpg/jpeg) and gt data (json/jsonl/txt),
    and it will be converted into input_tensor(vectorized image) and input_ids(tokenized string).
    Args:
        dataset_name_or_path: name of dataset (available at huggingface.co/datasets) or the path containing image files and metadata.jsonl
        max_length: the max number of tokens for the target sequences
        split: whether to load "train", "validation" or "test" split
        ignore_id: ignore_index for torch.nn.CrossEntropyLoss
        task_start_token: the special token to be fed to the decoder to conduct the target task
        prompt_end_token: the special token at the end of the sequences
        sort_json_key: whether or not to sort the JSON keys
    """

    def __init__(
        self,
        dataset_name_or_path: str,
        max_length: int,
        split: str = "train",
        ignore_id: int = -100,
        task_start_token: str = "<s>",
        prompt_end_token: str = None,
        sort_json_key: bool = True,
    ):
        super().__init__()

        self.max_length = max_length
        self.split = split
        self.ignore_id = ignore_id
        self.task_start_token = task_start_token
        self.prompt_end_token = prompt_end_token if prompt_end_token else task_start_token
        self.sort_json_key = sort_json_key

        self.dataset = load_dataset(dataset_name_or_path, split=self.split)
        self.dataset_length = len(self.dataset)

        self.gt_token_sequences = []
        for sample in self.dataset:
            ground_truth = json.loads(sample["ground_truth"])
            if "gt_parses" in ground_truth:  # when multiple ground truths are available, e.g., docvqa
                assert isinstance(ground_truth["gt_parses"], list)
                gt_jsons = ground_truth["gt_parses"]
            else:
                assert "gt_parse" in ground_truth and isinstance(ground_truth["gt_parse"], dict)
                gt_jsons = [ground_truth["gt_parse"]]

            self.gt_token_sequences.append(
                [
                    self.json2token(
                        gt_json,
                        update_special_tokens_for_json_key=self.split == "train",
                        sort_json_key=self.sort_json_key,
                    )
                    + processor.tokenizer.eos_token
                    for gt_json in gt_jsons  # load json from list of json
                ]
            )

        self.add_tokens([self.task_start_token, self.prompt_end_token])
        self.prompt_end_token_id = processor.tokenizer.convert_tokens_to_ids(self.prompt_end_token)

    def json2token(self, obj: Any, update_special_tokens_for_json_key: bool = True, sort_json_key: bool = True):
        """
        Convert an ordered JSON object into a token sequence
        """
        if type(obj) == dict:
            if len(obj) == 1 and "text_sequence" in obj:
                return obj["text_sequence"]
            else:
                output = ""
                if sort_json_key:
                    keys = sorted(obj.keys(), reverse=True)
                else:
                    keys = obj.keys()
                for k in keys:
                    if update_special_tokens_for_json_key:
                        self.add_tokens([fr"<s_{k}>", fr"</s_{k}>"])
                    output += (
                        fr"<s_{k}>"
                        + self.json2token(obj[k], update_special_tokens_for_json_key, sort_json_key)
                        + fr"</s_{k}>"
                    )
                return output
        elif type(obj) == list:
            return r"<sep/>".join(
                [self.json2token(item, update_special_tokens_for_json_key, sort_json_key) for item in obj]
            )
        else:
            obj = str(obj)
            if f"<{obj}/>" in added_tokens:
                obj = f"<{obj}/>"  # for categorical special tokens
            return obj

    def add_tokens(self, list_of_tokens: List[str]):
        """
        Add special tokens to tokenizer and resize the token embeddings of the decoder
        """
        newly_added_num = processor.tokenizer.add_tokens(list_of_tokens)
        if newly_added_num > 0:
            model.decoder.resize_token_embeddings(len(processor.tokenizer))
            added_tokens.extend(list_of_tokens)

    def __len__(self) -> int:
        return self.dataset_length

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Load image from image_path of given dataset_path and convert into input_tensor and labels
        Convert gt data into input_ids (tokenized string)
        Returns:
            input_tensor : preprocessed image
            input_ids : tokenized gt_data
            labels : masked labels (model doesn't need to predict prompt and pad token)
        """
        sample = self.dataset[idx]

        # inputs
        pixel_values = processor(sample["image"], random_padding=self.split == "train", return_tensors="pt").pixel_values
        pixel_values = pixel_values.squeeze()

        # targets
        target_sequence = random.choice(self.gt_token_sequences[idx])  # can be more than one, e.g., DocVQA Task 1
        input_ids = processor.tokenizer(
            target_sequence,
            add_special_tokens=False,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token
        # labels[: torch.nonzero(labels == self.prompt_end_token_id).sum() + 1] = self.ignore_id  # model doesn't need to predict prompt (for VQA)
        return pixel_values, labels, target_sequence

Next, we instantiate the datasets:

In [ ]:
# we update some settings which differ from pretraining; namely the size of the images + no rotation required
# source: https://github.com/clovaai/donut/blob/master/config/train_cord.yaml
from datasets import load_dataset
processor.feature_extractor.size = image_size[::-1] # should be (width, height)
processor.feature_extractor.do_align_long_axis = False

train_dataset = DonutDataset("/content/dataset", max_length=max_length,
                             split="train", task_start_token="<s_cord-v2>", prompt_end_token="<s_cord-v2>",
                             sort_json_key=False, # cord dataset is preprocessed, so no need for this
                             )

val_dataset = DonutDataset("/content/dataset", max_length=max_length,
                             split="validation", task_start_token="<s_cord-v2>", prompt_end_token="<s_cord-v2>",
                             sort_json_key=False, # cord dataset is preprocessed, so no need for this
                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/donut/processing_donut.py:186: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /root/.cache/huggingface/datasets/imagefolder/dataset-84857b314cbba886/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

In [ ]:
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(['<s_cord-v2>'])[0]

Next, we create corresponding PyTorch DataLoaders, which allow us to loop over the dataset in batches:

In [ ]:
from torch.utils.data import DataLoader

# feel free to increase the batch size if you have a lot of memory
# I'm fine-tuning on Colab and given the large image size, batch size > 1 is not feasible
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=2)

## Define LightningModule

In [ ]:
from pathlib import Path
import re
from nltk import edit_distance
import numpy as np
import math

from torch.nn.utils.rnn import pad_sequence
from torch.optim.lr_scheduler import LambdaLR

import pytorch_lightning as pl
from pytorch_lightning.utilities import rank_zero_only


class DonutModelPLModule(pl.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

    def training_step(self, batch, batch_idx):
        pixel_values, labels, _ = batch

        outputs = self.model(pixel_values, labels=labels)
        loss = outputs.loss
        self.log_dict({"train_loss": loss}, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        pixel_values, labels, answers = batch
        batch_size = pixel_values.shape[0]
        # we feed the prompt to the model
        decoder_input_ids = torch.full((batch_size, 1), self.model.config.decoder_start_token_id, device=self.device)

        outputs = self.model.generate(pixel_values,
                                   decoder_input_ids=decoder_input_ids,
                                   max_length=max_length,
                                   early_stopping=True,
                                   pad_token_id=self.processor.tokenizer.pad_token_id,
                                   eos_token_id=self.processor.tokenizer.eos_token_id,
                                   use_cache=True,
                                   num_beams=1,
                                   bad_words_ids=[[self.processor.tokenizer.unk_token_id]],
                                   return_dict_in_generate=True,)

        predictions = []
        for seq in self.processor.tokenizer.batch_decode(outputs.sequences):
            seq = seq.replace(self.processor.tokenizer.eos_token, "").replace(self.processor.tokenizer.pad_token, "")
            seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
            predictions.append(seq)

        scores = list()
        for pred, answer in zip(predictions, answers):
            pred = re.sub(r"(?:(?<=>) | (?=</s_))", "", pred)
            # NOT NEEDED ANYMORE
            # answer = re.sub(r"<.*?>", "", answer, count=1)
            answer = answer.replace(self.processor.tokenizer.eos_token, "")
            scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

            if self.config.get("verbose", False) and len(scores) == 1:
                print(f"Prediction: {pred}")
                print(f"    Answer: {answer}")
                print(f" Normed ED: {scores[0]}")

        return scores

    def validation_epoch_end(self, validation_step_outputs):
        # I set this to 1 manually
        # (previously set to len(self.config.dataset_name_or_paths))
        num_of_loaders = 1
        if num_of_loaders == 1:
            validation_step_outputs = [validation_step_outputs]
        assert len(validation_step_outputs) == num_of_loaders
        cnt = [0] * num_of_loaders
        total_metric = [0] * num_of_loaders
        val_metric = [0] * num_of_loaders
        for i, results in enumerate(validation_step_outputs):
            for scores in results:
                cnt[i] += len(scores)
                total_metric[i] += np.sum(scores)
            val_metric[i] = total_metric[i] / cnt[i]
            val_metric_name = f"val_metric_{i}th_dataset"
            self.log_dict({val_metric_name: val_metric[i]}, sync_dist=True)
        self.log_dict({"val_metric": np.sum(total_metric) / np.sum(cnt)}, sync_dist=True)

    def configure_optimizers(self):
        # TODO add scheduler
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config.get("lr"))

        return optimizer

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return val_dataloader

## Train

In [ ]:
config = {"max_epochs":20,
          "val_check_interval":1.0, # how many times we want to validate during an epoch
          "check_val_every_n_epoch":1,
          "gradient_clip_val":1.0,
          "num_training_samples_per_epoch": 1000,
          "lr":1e-5,
          "train_batch_sizes": [8],
          "val_batch_sizes": [1],
          # "seed":2022,
          "num_nodes": 1,
          "warmup_steps": 300, # 800/8*30/10, 10%
          "result_path": "./result",
          "verbose": True,
          }

model_module = DonutModelPLModule(config, processor, model)

In [ ]:
!huggingface-cli login
#(Copy token to clipboard)
#right click to paste (trust that it pastes, it won’t show) OR ctrl-shift-V into terminal to paste token
#(hit ENTER)

In [ ]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import Callback

wandb_logger = WandbLogger(project="Donut", name="to-be/DonutGhega")

class PushToHubCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        print(f"Pushing model to the hub, epoch {trainer.current_epoch}")
        pl_module.model.push_to_hub("to-be/DonutGhega", commit_message=f"Training in progress, epoch {trainer.current_epoch}")

    def on_train_end(self, trainer, pl_module):
        print(f"Pushing model to the hub after training")
        pl_module.processor.push_to_hub("to-be/DonutGhega", commit_message=f"Training done")
        pl_module.model.push_to_hub("to-be/DonutGhega", commit_message=f"Training done")

    def on_train_start(self, trainer, pl_module):
        print(f"Pushing processor config to the hub when starting training")
        pl_module.processor.push_to_hub("to-be/DonutGhega", commit_message=f"Training started")

trainer = pl.Trainer(
        accelerator="gpu",
        devices=1,
        max_epochs=config.get("max_epochs"),
        #val_check_interval=config.get("val_check_interval"),
        #check_val_every_n_epoch=config.get("check_val_every_n_epoch"),
        gradient_clip_val=config.get("gradient_clip_val"),
        precision=16, # we'll use mixed precision
        num_sanity_val_steps=0,
        logger=wandb_logger,
        callbacks=[PushToHubCallback()],
)

trainer.fit(model_module)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                      | Params
----------------------------------------------------
0 | model | VisionEncoderDecoderModel | 201 M 
----------------------------------------------------
201 M     Trainable params
0         Non-trainable params
201 M     Total params
403.743   Total estimated model params size (MB)


Pushing processor config to the hub when starting training


/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'preprocessor_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'added_tokens.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr

...

Prediction: <s_DocType>patent</s_DocType><s_FilingDate>17. Mai 2002 (17.05.2002)</s_FilingDate><s_Classification>B60R 21/01</s_Classification><s_PublicationDate>28. November 2002 (28.11.2002)</s_PublicationDate><s_ApplicationNumber>PCT/AT02/00152</s_ApplicationNumber>
    Answer: <s_DocType>patent</s_DocType><s_FilingDate>17. Mai 2002 (17.05.2002)</s_FilingDate><s_Classification>B60R 21/01</s_Classification><s_PublicationDate>28. November 2002 (28.11.2002)</s_PublicationDate><s_ApplicationNumber>PCT/AT02/00152</s_ApplicationNumber>
 Normed ED: 0.0
Prediction: <s_DocType>patent</s_DocType><s_FilingDate>10..05.2002</s_FilingDate><s_RepresentiveFL>Patentna pisarna d.0.0., Copova 14 p.p. 1725, 1001 Ljubljana, Sl</s_RepresentiveFL><s_Classification>A47J 31/40</s_Classification><s_PublicationDate>30.04.2005</s_PublicationDate><s_ApplicationNumber>10.05.2002 EP 02010526.8</s_ApplicationNumber>
    Answer: <s_DocType>patent</s_DocType><s_FilingDate>10.05.2002</s_FilingDate><s_RepresentiveFL>Pa

pytorch_model.bin:   0%|          | 0.00/809M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/809M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## Evaluate

After training, we can evaluate the model on the validation set.

to avoid *!pip install -q donut-python* throwing error:
*NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968*

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -q donut-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 48.1 MB/s eta 0:00:00


In [ ]:
import re
import json
import torch
import codecs
import io
import os
from PIL import Image
from tqdm.auto import tqdm
import numpy as np

from donut import JSONParseEvaluator



device = "cuda" if torch.cuda.is_available() else "cpu"

model.eval()
model.to(device)

output_list = []
accs = []



tableAll = []
headers = ["recognition","test image"]
#statistics dictionaries
nr_of_times_in_testset = {}
nr_of_times_in_testset_matching_gt = {}
nr_of_times_in_testset_false_positive = {}
nr_of_docs_everything_found = 0
with open("/content/dataset/validation/metadata.jsonl", 'r', encoding='utf-8-sig') as fp:
    Lines = fp.readlines()
    for line in Lines:
        #read line from metadata.jsonl from test folder in dataset with stripping BOM
        # prepare encoder inputs
        print(line)
        linedict = json.loads(line) # line is str
        filename = linedict['file_name']
        imglink = '<img style="max-width:1000; max-height:1100;" src="' + filename + '"/>'
        gt = linedict['ground_truth']  # becomes str
        gt_dict = json.loads(gt) #convert to dict
        gtresult = gt_dict['gt_parse'] #dict
        #make predictions using filename and model
        img = Image.open("/content/dataset/validation/" + filename).convert("RGB")



        pixel_values = processor(img, return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        # prepare decoder inputs
        task_prompt = "<s_cord-v2>"
        decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
        decoder_input_ids = decoder_input_ids.to(device)

        # autoregressively generate sequence
        outputs = model.generate(
                pixel_values,
                decoder_input_ids=decoder_input_ids,
                max_length=model.decoder.config.max_position_embeddings,
                early_stopping=True,
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
                use_cache=True,
                num_beams=1,
                bad_words_ids=[[processor.tokenizer.unk_token_id]],
                return_dict_in_generate=True,
            )
        # turn into JSON
        seq = processor.batch_decode(outputs.sequences)[0]
        seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
        seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
        seq = processor.token2json(seq)

        predictions_json = seq
        print(str(predictions_json))
        # result = predictions_json['predictions'] #list
        #result = result[0]  #dict
        output = ''
        everythingfoundindoc = True
        for key, value in seq.items():
          #print(key, value)
          #print(str(result))
          try:
            nr_of_times_in_testset[key] = nr_of_times_in_testset.get(key, 0) + 1 #increment this item in dict or add if it doesn't exist yet
            if value != '' and value == gtresult.get(key, ''):
              newline = '<div style="background-color:lightgreen;color:black">' #found what ground truth says
              nr_of_times_in_testset_matching_gt[key] = nr_of_times_in_testset_matching_gt.get(key, 0) + 1
            elif value != '' and gtresult.get(key, '') == '':
              newline = '<div style="background-color:yellow;color:black">' #found more than ground truth says
              everythingfoundindoc = False
            elif value == '' and gtresult.get(key, '') == '':
              newline = '<div style="background-color:grey;color:black">' #nothing found but there is nothing in gt
            else:
              newline = '<div style="background-color:salmon;color:black">'
              nr_of_times_in_testset_false_positive[key] = nr_of_times_in_testset_false_positive.get(key, 0) + 1
              everythingfoundindoc = False
            newline = newline + key + '&ensp;' + value + '&ensp;' + '(gt: ' + gtresult.get(key,'') + ' )' + '<BR>' + '</div>'
            output = output + newline
          except:
            newline = '<div style="background-color:blue;color:black">'
            everythingfoundindoc = False
            newline = newline + key + '&ensp;' + 'PROBLEM with output of model' + '&ensp;' + '(gt: ' + gtresult.get(key,'') + ' )' + '<BR>' + '</div>'
            output = output + newline
        #put output in between TD elements in HTML
        tableAll.append([output,imglink])
        if everythingfoundindoc == True:
          nr_of_docs_everything_found += 1

nr_of_docs = len(tableAll)
nr_of_docs_everything_found_percentage = str(round(nr_of_docs_everything_found / nr_of_docs * 100, 2))

{"file_name": "document-001-106931.in.000.png", "ground_truth": "{\"gt_parse\": { \"DocType\": \"patent\", \"FilingDate\": \"02.12.2008\", \"RepresentiveFL\": \"Kiehl, Hubert etal\", \"Classification\": \"A47J 31/40 (26)\", \"PublicationDate\": \"10.06.2009\", \"ApplicationNumber\": \"08356148.0\" } }"}

{'DocType': 'patent', 'FilingDate': '02.12.2008', 'RepresentiveFL': 'Kiehl, Hubert etal', 'Classification': 'A47J 31/40 (26)', 'PublicationDate': '10.06.2009', 'ApplicationNumber': '08356148.0'}
{"file_name": "document-009-108190.in.000.png", "ground_truth": "{\"gt_parse\": { \"DocType\": \"patent\", \"FilingDate\": \"23. Dezember 2003 (23.12.2003)\", \"Classification\": \"A63C 5/035,\", \"PublicationDate\": \"22. Juli 2004 (22.07.2004)\", \"ApplicationNumber\": \"PCT/AT2003/000387\" } }"}

{'DocType': 'patent', 'FilingDate': '23. Dczcmbci 2003 (23.12.2003)', 'Classification': 'A63C 5/035,', 'PublicationDate': '22. Juli 2004 (22.07.2004)', 'ApplicationNumber': 'PCT/AT2003/000387'}
{"fi

write a html file

In [ ]:
import os
!pip install tabulate
from tabulate import tabulate
#write HTML to test folder in dataset
os.chdir("/content/")
with open('result' + '_report.html', 'w',encoding="utf-8") as f:
  f.write('<h1>All test results: ' + '</h1>')
  f.write('<h2>Total number of docs: ' + str(nr_of_docs) + ' </h2>')
  f.write('<h2>Number of documents completely recognised: ' + str(nr_of_docs_everything_found) + ' = ' + nr_of_docs_everything_found_percentage + '%</h2>')
  f.write('<h1>Field info: ' + '</h1>')
  for key, value in nr_of_times_in_testset.items():
    f.write('<h2>Number of times field: ' + key + ' in testset: ' + str(value) + '. Correctly found: '+ str(round(nr_of_times_in_testset_matching_gt.get(key,0) / value * 100, 2)) + '%. False Positives: '+ str(round(nr_of_times_in_testset_false_positive.get(key,0) / value * 100, 2)) +' %.</h2>')
  f.write(tabulate(tableAll, headers, tablefmt="unsafehtml"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
